In [42]:
from numpy import vstack
from numpy import sqrt
import pandas as pd
from sklearn.metrics import mean_squared_error
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import Tensor
from torch.nn import Linear
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
from torch.nn import MSELoss
from torch.nn.init import xavier_uniform_
from tqdm import tqdm


In [60]:
class CSVDataset(Dataset):
    # load the dataset
    def __init__(self, path):
        # load the csv file as a dataframe
        df = pd.read_csv(path, header=None)
        df.head()
        print(f"NaN values in dataset:\n{df.isna().sum()}")

        
        df.drop(columns=df.columns[-1],inplace=True)
        df.fillna(df.mean(),inplace=True)
        print(f"NaN values in dataset:\n{df.isna().sum()}")
        # store the inputs and outputs
        self.X = df.values[:, :-1].astype('float32')
        self.y = df.values[:, -1].astype('float32')
        # ensure target has the right shape
        self.y = self.y.reshape((len(self.y), 1))
    # number of rows in the dataset
    def __len__(self):
        return len(self.X)
    # get a row at an index
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]
    # get indexes for train and test rows
    def get_splits(self, n_test=0.33):
    # determine sizes
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        # calculate the split
        return random_split(self, [train_size, test_size])


In [44]:
# model definition
class MLP(Module):
    # define model elements
    def __init__(self, n_inputs):
        super(MLP, self).__init__()
        # input to first hidden layer
        self.hidden1 = Linear(n_inputs, 10)
        xavier_uniform_(self.hidden1.weight)
        self.act1 = Sigmoid()
        # second hidden layer
        self.hidden2 = Linear(10, 8)
        xavier_uniform_(self.hidden2.weight)
        self.act2 = Sigmoid()
        # third hidden layer and output
        self.hidden3 = Linear(8, 1)
        xavier_uniform_(self.hidden3.weight)
    # forward propagate input
    def forward(self, X):
    # input to first hidden layer
        X = self.hidden1(X)
        X = self.act1(X)
        # second hidden layer
        X = self.hidden2(X)
        X = self.act2(X)
        # third hidden layer and output
        X = self.hidden3(X)
        return X


In [45]:
# prepare the dataset
def prepare_data(path):
    # load the dataset
    dataset = CSVDataset(path)
    # calculate split
    train, test = dataset.get_splits()
    # prepare data loaders
    train_dl = DataLoader(train, batch_size=32, shuffle=True)
    test_dl = DataLoader(test, batch_size=1024, shuffle=False)
    return train_dl, test_dl


In [56]:
# train the model
def train_model(train_dl, model):
    size = len(train_dl.dataset)
    # define the optimization
    criterion = MSELoss()
    optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
    # enumerate epochs
    for epoch in tqdm(range(10),desc='Training Epochs'):
        print(f"Epoch {epoch+1}\n-------------------------------")
        # enumerate mini batches
        for batch, (inputs, targets) in enumerate(train_dl):
        # clear the gradients
            optimizer.zero_grad()
            # compute the model output
            yhat = model(inputs)
            # calculate loss
            loss = criterion(yhat, targets)
            # credit assignment
            loss.backward()
            # update model weights
            optimizer.step()
            #if batch % 100 == 0:
            loss, current = loss.item(), batch * len(inputs)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [47]:
# evaluate the model
def evaluate_model(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        # evaluate the model on the test set
        yhat = model(inputs)
        # retrieve numpy array
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        actual = actual.reshape((len(actual), 1))
        # store
        predictions.append(yhat)
        actuals.append(actual)
        predictions, actuals = vstack(predictions), vstack(actuals)
        # calculate mse
        mse = mean_squared_error(actuals, predictions)
        return mse

In [48]:
# make a class prediction for one row of data
def predict(row, model):
    # convert row to data
    row = Tensor([row])
    # make prediction
    yhat = model(row)
    # retrieve numpy array
    yhat = yhat.detach().numpy()
    return yhat

In [ ]:
# prepare the data
path = 'housing.csv'
train_dl, test_dl = prepare_data(path)
print(len(train_dl.dataset), len(test_dl.dataset))
# Define the network (9 input features)
model = MLP(8)
# train the model
train_model(train_dl, model)
# evaluate the model
mse = evaluate_model(test_dl, model)
print('MSE: %.3f, RMSE: %.3f' % (mse, sqrt(mse)))
# make a single prediction (expect class=1)
row = [-122.23, 37.88, 41.0, 880.0, 129.0, 322.0, 126.0, 8.3252, 452600.0]
row = row[:-1]
yhat = predict(row, model)



NaN values in dataset:
0      0
1      0
2      0
3      0
4    207
5      0
6      0
7      0
8      0
9      0
dtype: int64
NaN values in dataset:
0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
dtype: int64
13829 6811


Training Epochs:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1
-------------------------------
loss: 58445844480.000000 [    0/13829]
loss: 58756952064.000000 [   32/13829]
loss: 49175085056.000000 [   64/13829]
loss: 21832314880.000000 [   96/13829]
loss: 13927940096.000000 [  128/13829]
loss: 9667042304.000000 [  160/13829]
loss: 16880727040.000000 [  192/13829]
loss: 24215590912.000000 [  224/13829]
loss: 20771411968.000000 [  256/13829]
loss: 22069252096.000000 [  288/13829]
loss: 25883103232.000000 [  320/13829]
loss: 23459500032.000000 [  352/13829]
loss: 16909578240.000000 [  384/13829]
loss: 16485473280.000000 [  416/13829]
loss: 10385342464.000000 [  448/13829]
loss: 18300325888.000000 [  480/13829]
loss: 13205173248.000000 [  512/13829]
loss: 10481135616.000000 [  544/13829]
loss: 20776132608.000000 [  576/13829]
loss: 16889994240.000000 [  608/13829]
loss: 13411224576.000000 [  640/13829]
loss: 25637085184.000000 [  672/13829]
loss: 10888257536.000000 [  704/13829]
loss: 13044611072.000000 [  736/13829]
loss: 14838656000.000000 

Training Epochs:  10%|█         | 1/10 [00:00<00:08,  1.04it/s]

loss: 14813077504.000000 [11392/13829]
loss: 10133614592.000000 [11424/13829]
loss: 14197272576.000000 [11456/13829]
loss: 11058464768.000000 [11488/13829]
loss: 13270936576.000000 [11520/13829]
loss: 14760569856.000000 [11552/13829]
loss: 14664664064.000000 [11584/13829]
loss: 14355054592.000000 [11616/13829]
loss: 4729939968.000000 [11648/13829]
loss: 18264786944.000000 [11680/13829]
loss: 14016573440.000000 [11712/13829]
loss: 15680176128.000000 [11744/13829]
loss: 8239694848.000000 [11776/13829]
loss: 16662550528.000000 [11808/13829]
loss: 11511795712.000000 [11840/13829]
loss: 15908998144.000000 [11872/13829]
loss: 8990536704.000000 [11904/13829]
loss: 23765860352.000000 [11936/13829]
loss: 11988040704.000000 [11968/13829]
loss: 14345454592.000000 [12000/13829]
loss: 15956282368.000000 [12032/13829]
loss: 13919546368.000000 [12064/13829]
loss: 12184610816.000000 [12096/13829]
loss: 14192173056.000000 [12128/13829]
loss: 16634264576.000000 [12160/13829]
loss: 9278938112.000000 [121

...

Training Epochs:  90%|█████████ | 9/10 [00:07<00:00,  1.25it/s]

loss: 11701068800.000000 [11680/13829]
loss: 14138400768.000000 [11712/13829]
loss: 13133489152.000000 [11744/13829]
loss: 6226119680.000000 [11776/13829]
loss: 8797878272.000000 [11808/13829]
loss: 13486545920.000000 [11840/13829]
loss: 14374870016.000000 [11872/13829]
loss: 12848339968.000000 [11904/13829]
loss: 8324617216.000000 [11936/13829]
loss: 11337392128.000000 [11968/13829]
loss: 11221817344.000000 [12000/13829]
loss: 26775939072.000000 [12032/13829]
loss: 11905390592.000000 [12064/13829]
loss: 13076622336.000000 [12096/13829]
loss: 12265935872.000000 [12128/13829]
loss: 15080985600.000000 [12160/13829]
loss: 19968591872.000000 [12192/13829]
loss: 11938800640.000000 [12224/13829]
loss: 17001680896.000000 [12256/13829]
loss: 10154424320.000000 [12288/13829]
loss: 16319968256.000000 [12320/13829]
loss: 15983692800.000000 [12352/13829]
loss: 11587842048.000000 [12384/13829]
loss: 14888496128.000000 [12416/13829]
loss: 10526733312.000000 [12448/13829]
loss: 18045786112.000000 [12

Training Epochs: 100%|██████████| 10/10 [00:08<00:00,  1.22it/s]

loss: 17137561600.000000 [13408/13829]
loss: 18324045824.000000 [13440/13829]
loss: 14189172736.000000 [13472/13829]
loss: 8792189952.000000 [13504/13829]
loss: 15834114048.000000 [13536/13829]
loss: 10998532096.000000 [13568/13829]
loss: 15507615744.000000 [13600/13829]
loss: 13696248832.000000 [13632/13829]
loss: 17030981632.000000 [13664/13829]
loss: 17085078528.000000 [13696/13829]
loss: 19411752960.000000 [13728/13829]
loss: 15470594048.000000 [13760/13829]
loss: 15058317312.000000 [13792/13829]
loss: 9932634112.000000 [ 2160/13829]
MSE: 13663270912.000, RMSE: 116889.995
Predicted: 228237.875



C:\Users\Diae\AppData\Local\Temp\ipykernel_2428\3466736938.py:16: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('Predicted: %.3f' % yhat)
